In [30]:
from google.colab import drive
drive.mount('./gdrive')

Mounted at ./gdrive


In [5]:
!pip install autocrop

     |████████████████████████████████| 174kB 7.9MB/s 
     |████████████████████████████████| 2.2MB 14.6MB/s 
  Created wheel for autocrop: filename=autocrop-1.1.1-cp37-none-any.whl size=167184 sha256=f76db0a9b031171c9f06a74674517dc82cdfd8366f7f9b46c22bd1425d9e3317
  Stored in directory: /root/.cache/pip/wheels/6d/54/50/6286b6aa3bd202b188d97d293949a021a937ebb65bd71844fa
Successfully built autocrop
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


In [1]:
import cv2
import numpy as np
from PIL import Image
import dlib
import os
from autocrop import Cropper


In [13]:
#Alinhador

def draw_predict(frame, left, top, right, bottom):
    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 3)

def rotate_point(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + np.cos(angle) * (px - ox) - np.sin(angle) * (py - oy)
    qy = oy + np.sin(angle) * (px - ox) + np.cos(angle) * (py - oy)
    return qx, qy

def is_between(point1, point2, point3, extra_point):
    c1 = (point2[0] - point1[0]) * (extra_point[1] - point1[1]) - (point2[1] - point1[1]) * (extra_point[0] - point1[0])
    c2 = (point3[0] - point2[0]) * (extra_point[1] - point2[1]) - (point3[1] - point2[1]) * (extra_point[0] - point2[0])
    c3 = (point1[0] - point3[0]) * (extra_point[1] - point3[1]) - (point1[1] - point3[1]) * (extra_point[0] - point3[0])
    if (c1 < 0 and c2 < 0 and c3 < 0) or (c1 > 0 and c2 > 0 and c3 > 0):
        return True
    else:
        return False

def distance(a, b):
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def cosine_formula(length_line1, length_line2, length_line3):
    cos_a = -(length_line3 ** 2 - length_line2 ** 2 - length_line1 ** 2) / (2 * length_line2 * length_line1)
    return cos_a

def rotation_detection_dlib(title):
    img = cv2.imread(title)
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('./shape_predictor_5_face_landmarks.dat')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    if len(rects) > 0:
        for rect in rects:
            x = rect.left()
            y = rect.top()
            w = rect.right()
            h = rect.bottom()
            shape = predictor(gray, rect)
            shape_normal = []
            for i in range(0, 5):
                shape_normal.append((i, (shape.part(i).x, shape.part(i).y)))
            shape = shape_normal

            nose = shape[4][1]
            left_eye_x = int(shape[3][1][0] + shape[2][1][0]) // 2
            left_eye_y = int(shape[3][1][1] + shape[2][1][1]) // 2
            right_eyes_x = int(shape[1][1][0] + shape[0][1][0]) // 2
            right_eyes_y = int(shape[1][1][1] + shape[0][1][1]) // 2

            nose, left_eye, right_eye = nose, (left_eye_x, left_eye_y), (right_eyes_x, right_eyes_y)

            center_of_forehead = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
            center_pred = (int((x + w) / 2), int((y + y) / 2))
            length_line1 = distance(center_of_forehead, nose)
            length_line2 = distance(center_pred, nose)
            length_line3 = distance(center_pred, center_of_forehead)
            cos_a = cosine_formula(length_line1, length_line2, length_line3)
            angle = np.arccos(cos_a)
            rotated_point = rotate_point(nose, center_of_forehead, angle)
            rotated_point = (int(rotated_point[0]), int(rotated_point[1]))
            if is_between(nose, center_of_forehead, center_pred, rotated_point):
                angle = np.degrees(-angle)
            else:
                angle = np.degrees(angle)

            img = Image.fromarray(img)
            img = np.array(img.rotate(angle))

        return img
    else:
        return img

In [10]:
cropper = Cropper(width=150,height=150,
                  face_percent=100, )

In [97]:
def tratar_imagem(image, path='/content/', destiny_path='/content/'):
  #print('"' + path + image + '" Tratando...')
  img = rotation_detection_dlib(path + image)
  cropped_array = cropper.crop(img)
  if not(cropped_array is None):
    cropped_image = Image.fromarray(cropped_array)
    cropped_image.save(destiny_path + image)
    #print('"' + path + image + '" Tratado !')
  else:
     print('"' + path + image + '" Falhou em ser tratado!')

def tratar_pasta(folder, path='/content/' ,destiny_path='/content/'):
  os.mkdir(destiny_path + folder + '_tratado')

  for image in os.listdir(path + folder):
    if(image[len(image)-4:len(image)] == ".jpg"): 
      tratar_imagem(image, path=path+folder+'/',
                destiny_path=destiny_path + folder + '_tratado/')

def tratar_base(base, path='/content/', destiny_path='/content/'):
  os.mkdir(destiny_path + base + '_tratado')

  for folder in os.listdir(path + base):
    tratar_pasta(folder, path=path+base+'/',
                destiny_path=destiny_path+base+'_tratado/')
    

In [100]:
#sequência de comandos
#img = rotation_detection_dlib('./gdrive/Shareddrives/DB-LARC/face-base/asian/AF0309/AF0309_1100_NE.jpg')

#cropped_array = cropper.crop(img)

#cropped_image = Image.fromarray(cropped_array)
#cropped_image.save('/content/japa_final.jpg')


tratar_base('african-american', path='./gdrive/Shareddrives/DB-LARC/face-base/', destiny_path='./gdrive/Shareddrives/DB-LARC/face-base/')
tratar_base('caucasian', path='./gdrive/Shareddrives/DB-LARC/face-base/', destiny_path='./gdrive/Shareddrives/DB-LARC/face-base/')
tratar_base('hispanic', path='./gdrive/Shareddrives/DB-LARC/face-base/', destiny_path='./gdrive/Shareddrives/DB-LARC/face-base/')
tratar_base('multiracial', path='./gdrive/Shareddrives/DB-LARC/face-base/', destiny_path='./gdrive/Shareddrives/DB-LARC/face-base/')
#tratar_base('african-american', path='./gdrive/Shareddrives/DB-LARC/face-base/', destiny_path='./opora/')

#tratar_imagem('AF0309_1100_NE.jpg', path='./gdrive/Shareddrives/DB-LARC/face-base/asian/AF0309/')


"./gdrive/Shareddrives/DB-LARC/face-base/african-american/BF0618/BF0618_2201_15R.jpg" Falhou em ser tratado!
"./gdrive/Shareddrives/DB-LARC/face-base/african-american/BF0618/BF0618_2201_30R.jpg" Falhou em ser tratado!
"./gdrive/Shareddrives/DB-LARC/face-base/african-american/BF0618/BF0618_2201_30L.jpg" Falhou em ser tratado!
"./gdrive/Shareddrives/DB-LARC/face-base/african-american/BF0623/BF0623_2200_15L.jpg" Falhou em ser tratado!
"./gdrive/Shareddrives/DB-LARC/face-base/african-american/BF0623/BF0623_1100_15L.jpg" Falhou em ser tratado!
"./gdrive/Shareddrives/DB-LARC/face-base/african-american/BF0623/BF0623_1100_15R.jpg" Falhou em ser tratado!
"./gdrive/Shareddrives/DB-LARC/face-base/african-american/BF0623/BF0623_1100_30R.jpg" Falhou em ser tratado!
"./gdrive/Shareddrives/DB-LARC/face-base/african-american/BF0623/BF0623_2200_30L.jpg" Falhou em ser tratado!
"./gdrive/Shareddrives/DB-LARC/face-base/african-american/BF0623/BF0623_1100_30L.jpg" Falhou em ser tratado!
"./gdrive/Shareddri

In [94]:
!cp -R ./opora/asian_tratado ./gdrive/Shareddrives/DB-LARC/face-base